In [3]:
import os
import datetime
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
if 'current_dir' in locals():
    print("Directory has been set before:")
else:
    current_dir = os.getcwd()[:-9] 
print(current_dir)
os.chdir(current_dir)
submission = pd.read_csv(current_dir + 'submissions/submission_v1.csv') #submission as template (version is not important)

Directory has been set before:
/home/harry/Documents/customer-loyalty/


In [6]:
# Loading files for training and testing

# Load plain train data and test data:
data = pd.read_csv('all/train.csv')
testdata = pd.read_csv('all/test.csv')

# Load all features in the folder called 'features': (comment these lines out for custom set of features)
for f in listdir(current_dir + "/features"):
    if (f[:4] == "test"):
        print("this is a test feature : " + f)
        temp = pd.read_csv(current_dir + "/features/" + f)
        testdata.insert(testdata.shape[1], f[13:-4], temp[f[13:-4]] )
    if (f[:5] == "train"):
        print("this is a train feature : " + f)
        temp = pd.read_csv(current_dir + "/features/" + f)
        data.insert(data.shape[1]-1, f[14:-4], temp[f[14:-4]] )
        
# Load a custom set of features: (comment these lines out for all features)
# feature_names = ['test_feature_fam.csv','train_feature_fam.csv']

# for f in feature_names:
#     if (f[:4] == "test"):
#         print("this is a test feature : " + f)
#         temp = pd.read_csv(current_dir + "/features/" + f)
#         testdata.insert(testdata.shape[1], f[13:-4], temp[f[13:-4]] )
#     if (f[:5] == "train"):
#         print("this is a train feature : " + f)
#         temp = pd.read_csv(current_dir + "/features/" + f)
#         data.insert(data.shape[1]-1, f[14:-4], temp[f[14:-4]] )

this is a test feature : test_feature_fam.csv
this is a train feature : train_feature_fam.csv


In [7]:
print(testdata)

       first_active_month          card_id  feature_1  feature_2  feature_3  \
0                 2017-04  C_ID_0ab67a22ab          3          3          1   
1                 2017-01  C_ID_130fd0cbdd          2          3          0   
2                 2017-08  C_ID_b709037bc5          5          1          1   
3                 2017-12  C_ID_d27d835a9f          2          1          0   
4                 2015-12  C_ID_2b5e3df5c2          5          1          1   
5                 2017-07  C_ID_5814b4f13c          5          1          1   
6                 2017-10  C_ID_a1b3c75277          1          2          0   
7                 2017-02  C_ID_f7cada36d3          3          1          1   
8                 2017-07  C_ID_9d2bc8dfc4          4          3          0   
9                 2016-03  C_ID_6d8dba8475          5          1          1   
10                2017-03  C_ID_4859ac9ed5          5          2          1   
11                2017-10  C_ID_0e281e8ad6          

## Training
Training will be done in the following cells. First a random part of the training data is assigned as training set and the leftovers will be the validation set. By making the _trainsize_ variable round(train.shape[0]/1), the entirety of the training data is used as training set, so no evaluation will be done in the section _Testing within training set_.

In [8]:
train = np.array(data)
np.random.shuffle(train)
trainsize = round(train.shape[0]) #fraction of trainingset used for training the model
features = train.shape[1] - 3

# A part of the training set can be used as a test set,
# so the Root Mean Squared Error can be found
train_l = train[:trainsize,features+2]
train_x = train[:trainsize,2:features+2]

test_l = train[trainsize:,features+2]
test_x = train[trainsize:,2:features+2]


In [9]:
def CorrPrint(pandas_dataframe, numpy_array):
    corr_matrix = np.corrcoef((train[:,2:numpy_array.shape[1]].T).astype(float))
    
    print(" "*15, sep=' ', end='', flush=True)
    for i in range(numpy_array.shape[1]-2):
        print(pandas_dataframe.columns[i+2], " "*(11 - len(pandas_dataframe.columns[i+2])), sep=' ', end='', flush=True)
    print()
    for i in range(numpy_array.shape[1]-2):
        print(pandas_dataframe.columns[i+2], " "*(11 - len(pandas_dataframe.columns[i+2])), corr_matrix[i])
    return

CorrPrint(data,train)

               feature_1   feature_2   feature_3   fam         target      
feature_1    [ 1.         -0.13096861  0.58309179 -0.11721539 -0.01425105]
feature_2    [-0.13096861  1.          0.0609252  -0.15822695 -0.00624176]
feature_3    [ 0.58309179  0.0609252   1.         -0.19610544 -0.00812528]
fam          [-0.11721539 -0.15822695 -0.19610544  1.          0.05046633]
target       [-0.01425105 -0.00624176 -0.00812528  0.05046633  1.        ]


In [10]:
# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'xgboost'

In [11]:
# fit model no training data
model = XGBClassifier()
model.fit(train_x, train_l)

NameError: name 'XGBClassifier' is not defined

## Predictions for test set (creating a submission .csv file)
Make sure to use the entire training set in the the "Training" section

In [49]:
testset = np.array(testdata)
testset_x = testset[:,2:features+2]
# make predictions for test data
y_pred = model.predict(testset_x)
predictions = [round(value) for value in y_pred]
submission.target = predictions
# # evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [120]:
# submission.target = submission_predictions

for i in range(1,100,1):
    if os.path.isfile('submissions/submission_v0-00' + str(i) + '.csv') == False:
        submission.to_csv('submissions/submission_v0-00' + str(i) + '.csv', index = False)
        print('Submission was saved under the name:' + '\033[1m' ' submission_v0-00' + str(i) + '.csv' + '\033[0;0m')
        break

# [Errno 13] Permission denied: 'submissions/submission_v0-001.csv'
#        means that the file is still opened somewhere

Submission was saved under the name: submission_v0-004.csv


In [121]:
# run this after previous cel to see the submission
submission

,card_id,target
0,C_ID_0ab67a22ab,-0.358949
1,C_ID_130fd0cbdd,-0.411949
2,C_ID_b709037bc5,-0.292005
3,C_ID_d27d835a9f,-0.132738
4,C_ID_2b5e3df5c2,-0.689614
5,C_ID_5814b4f13c,-0.311885
6,C_ID_a1b3c75277,-0.166210
7,C_ID_f7cada36d3,-0.338184
8,C_ID_9d2bc8dfc4,-0.365771
9,C_ID_6d8dba8475,-0.629973
